This helper makes sure that the packages that rebuilt successfully with Python 3.10 but fail to install due to their dependencies not yet rebuilt have bugzillas blocked by the appropriate bugzillas of said dependencies.

 1. For every package that has been successfully rebuilt, we see if it has a bugzilla that blocks `PYTHON3.10` and `F35FailsToInstall`.
 2. If so, we parse the initial comment for missing `python3.10dist()` dependencies.
 3. We convert the dependencies to `python3.9dist()` dependencies and find their component.
 4. If the component has a `PYTHON3.10` bugzilla, we mark it as blocking the bugzilla from (1).

In [1]:
from IPython.display import display, Markdown

In [2]:
import bugzilla
bzapi = bugzilla.Bugzilla('bugzilla.redhat.com')
query = bzapi.build_query(product='Fedora')
query['blocks'] = 1890881  # PYTHON3.10
bugz = bzapi.query(query)

In [3]:
F35FailsToInstall = 1927313

In [4]:
python310 = !cat python310.pkgs

In [5]:
bugz = [b for b in bugz if b.status != 'CLOSED']

In [6]:
len(pure_fti_bugz := [b for b in bugz if b.component in python310 and F35FailsToInstall in b.blocks])

34

In [8]:
for bug in pure_fti_bugz:
    display(Markdown(f'# {bug.component}\n[rbz#{bug.id}](https://bugzilla.redhat.com/{bug.id})'))
    comments = bug.getcomments()
    initial_comment = comments[0]
    if initial_comment['creator'] != 'mhroncok@redhat.com':
        print('Not an automated FTI bug by Miro')
        continue

    blocked_by = set()
    lines = initial_comment['text'].splitlines()
    for line in lines:
        if line.startswith('  - nothing provides '):
            print(line)
            provide = line.split(' ')[5]
            if provide.startswith('('):
                provide = provide[1:]
            if provide.startswith('python3.10dist('):
                old_provide = provide.replace('python3.10dist(', 'python3.9dist(')
                blocker_components = !repoquery -q --repo=rawhide --source --whatprovides "{old_provide}" | pkgname
                if len(set(blocker_components)) == 1:
                    blocker_component = blocker_components[0]
                    blocker_bugz = [b for b in bugz if b.component == blocker_component]
                    if len(blocker_bugz) > 1:
                        print(f"    = too many bugzillas for {blocker_component}")
                        for bug in blocker_bugz:
                            print(f"        https://bugzilla.redhat.com/{bug.id}")
                    if blocker_bugz:
                        bid = blocker_bugz[0].id
                        print(f"    = blocked by {blocker_component} https://bugzilla.redhat.com/{bid}")
                        blocked_by.add(bid)
                else:
                    print('Check:', blocker_components)
    if blocked_by:
        add_blockers = bzapi.build_update(depends_on_add=sorted(blocked_by))
        bzapi.update_bugs([bug.id], add_blockers)

# androwarn
[rbz#1968774](https://bugzilla.redhat.com/1968774)

  - nothing provides python3.10dist(androguard) >= 3.2.1 needed by androwarn-1.6.1-7.fc35.noarch
    = blocked by androguard https://bugzilla.redhat.com/1968773


# ansible-lint
[rbz#1968775](https://bugzilla.redhat.com/1968775)

  - nothing provides python3.10dist(tenacity) needed by python3-ansible-lint-1:5.0.11-2.fc35.noarch
  - nothing provides python3.10dist(wcmatch) >= 7 needed by python3-ansible-lint-1:5.0.11-2.fc35.noarch
    = blocked by python-wcmatch https://bugzilla.redhat.com/1958908


# cobbler
[rbz#1968786](https://bugzilla.redhat.com/1968786)

...

  - nothing provides (python3.10dist(configargparse) < 2 with python3.10dist(configargparse) > 0.15) needed by rofimoji-5.1.0-6.fc35.noarch
    = blocked by python-configargparse https://bugzilla.redhat.com/1914818


# sgmanager
[rbz#1969188](https://bugzilla.redhat.com/1969188)

  - nothing provides python3.10dist(openstacksdk) needed by sgmanager-2.0.0-9+201801213git+146.861aa67.fc35.noarch
    = blocked by python-openstacksdk https://bugzilla.redhat.com/1969068


# zezere
[rbz#1969203](https://bugzilla.redhat.com/1969203)

  - nothing provides python3.10dist(mod-wsgi) needed by zezere-0.5-8.fc35.noarch
    = blocked by mod_wsgi https://bugzilla.redhat.com/1898158
